<a href="https://colab.research.google.com/github/atharvajoshi10/FederatedML/blob/dev%2Funified_fine_tuning/Roberta_Base_Unified_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base Model

Attempting to recreate the validation F1 of the following model.
https://huggingface.co/JeremiahZ/roberta-base-mrpc

# Dependencies

GPU - T4

Note : Force Update accelerate since it defaults to the old version.
Restart the notebook once the below cell has executed.

In [1]:
%%capture
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install accelerate -U

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, AutoConfig, MobileBertConfig, TrainingArguments, DataCollatorWithPadding
import numpy as np
import evaluate

# Import the Base Roberta Model


In [3]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenizer Definition

Since this a sequence classification job to check whether both the sentences say the same thing, we must tokenize every pair of sentence together.

In [4]:
raw_datasets = load_dataset("glue", "mrpc")

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"])


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training Parameters

Using custom training parameters along with computed validation metrics.

In [5]:
training_args = TrainingArguments("estimate_trainer", num_train_epochs=5, learning_rate=2e-05,
                                  lr_scheduler_type="linear", per_device_train_batch_size=16, seed=42,
                                  per_device_eval_batch_size=8, warmup_ratio=0.06)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Model Training

In [6]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.418000
1000,0.166100


TrainOutput(global_step=1150, training_loss=0.2668577243970788, metrics={'train_runtime': 306.1987, 'train_samples_per_second': 59.896, 'train_steps_per_second': 3.756, 'total_flos': 720461791784400.0, 'train_loss': 0.2668577243970788, 'epoch': 5.0})

# Checking the Validation F1

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])

preds = np.argmax(predictions.predictions, axis=-1)


metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8774509803921569, 'f1': 0.910394265232975}